In [72]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


In [54]:
df=pd.read_csv('sales.csv')

In [55]:
df.sample(5)

,Row ID,Order ID,Order Date Fa,Ship Date Fa,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
5236,5237,US-2021-121671,1401/08/15,1401/08/20,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Springfield,...,65807,Central,OFF-ST-10002344,Office Supplies,Storage,"Carina 42""Hx23 3/4""W Media Storage Unit",242.94,3,0.0,4.8588
4361,4362,US-2021-143819,1401/03/31,1401/04/04,Standard Class,KD-16270,Karen Daniels,Consumer,United States,Yonkers,...,10701,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,362.92,2,0.0,105.2468
6462,6463,US-2021-104689,1402/01/03,1402/01/07,Standard Class,FH-14365,Fred Hopkins,Corporate,United States,Los Angeles,...,90004,West,OFF-AR-10001149,Office Supplies,Art,"Sanford Colorific Colored Pencils, 12/Box",23.04,8,0.0,6.9120
9673,9674,US-2022-132185,1402/12/28,1403/01/04,Standard Class,DB-13615,Doug Bickford,Consumer,United States,Asheville,...,28806,South,OFF-AR-10000422,Office Supplies,Art,Pencil and Crayon Sharpener,8.76,5,0.2,0.7665
1657,1658,US-2019-102274,1399/12/22,1399/12/27,Standard Class,DH-13075,Dave Hallsten,Corporate,United States,Richmond,...,40475,South,TEC-PH-10002923,Technology,Phones,Logitech B530 USB Headset - headset - Full siz...,36.99,1,0.0,9.9873


In [56]:
df.duplicated().sum()

0

In [57]:
df1 = df.sort_values(by='Order ID', ascending=True)[['Order ID', 'Customer ID', 'Product ID', 'Product Name', 'Category']]
df_order_count = df1['Order ID'].value_counts()  

# Filter Order IDs that appear at least 3 times  
order_ids_to_keep = df_order_count[df_order_count >= 3].index  

# Filter the DataFrame to only include these Order IDs  
filtered_df1 = df1[df1['Order ID'].isin(order_ids_to_keep)] 

In [168]:
# Get unique products  
df_product = filtered_df1[['Product ID', 'Product Name', 'Category']].drop_duplicates('Product ID').reset_index(drop=True)  

# Encode data  
cols = ['Order ID', 'Product ID']  
df_encode = filtered_df1[cols].copy()  

# Create a LabelEncoder instance  
encoder = LabelEncoder()  

# Fit the encoder on the unique product IDs found in the filtered DataFrame  
df_product['Product code'] = encoder.fit_transform(df_product['Product ID'])
df_product['Category code'] = encoder.fit_transform(df_product['Category'])
df_encode = pd.merge(df_encode, df_product, on='Product ID', how='inner')
# Create a list of orders  
orders = df_encode.groupby('Order ID')['Product code'].apply(list).reset_index()  

X = []
y = []
for index, row in orders.iterrows():
    pl = row['Product code']
    for i in range(len(pl)):
        for j in range(len(pl)):
            if j > i:
                for k in range(len(pl)):
                    if k != i and k != j:
                        y.append(pl[k]) 
                        X.append([pl[i], pl[j]])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = KNeighborsClassifier(n_neighbors=5, weights='distance')
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.004190321708569884


In [169]:
def find_product(p_code):
    prod = df_product.query(f'`Product code` == {p_code}')
    return prod['Product ID'].values[0], prod['Product Name'].values[0], prod['Category'].values[0]

def recommendation(x1, x2):
    x = np.array([x1, x2]).reshape(1,-1)
    y = model.predict(x)
    return y

x1, x2 = X[np.random.choice(len(X_test), 1)][0]  #np.random.choice(y, 2)
y_pred = recommendation(x1, x2)
print(f'First product:       {find_product(x1)}')
print(f'Second product:      {find_product(x2)}')
print(f'Recommended product: {find_product(y_pred)}')
print(f'Product code prediction: {y_pred}')
print(f'Product codes refers:    {y[np.where((X[:,0] == x1) & (X[:,1] == x2))]}')


First product:       ('OFF-BI-10000050', 'Angle-D Binders with Locking Rings, Label Holders', 'Office Supplies')
Second product:      ('TEC-AC-10002718', 'Belkin Standard 104 key USB Keyboard', 'Technology')
Recommended product: ('OFF-AR-10003179', 'Dixon Ticonderoga Core-Lock Colored Pencils', 'Office Supplies')
Product code prediction: [526]
Product codes refers:    [ 338 1559  526]
